In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [5]:
data = pd.read_csv('../datasets/timeSeries1s.csv', header=0, index_col=0, low_memory=False)
gossip = pd.DataFrame(data)

gossip.head(10)

,identifier,experiment,type,_time,count
0,0,618,11.0,2023-11-16 21:19:03+00:00,76
1,0,618,11.0,2023-11-16 21:19:04+00:00,1390
2,0,618,11.0,2023-11-16 21:19:07+00:00,14
3,0,618,12.0,2023-11-16 21:19:04+00:00,7
4,0,618,12.0,2023-11-16 21:19:06+00:00,82
5,0,618,12.0,2023-11-16 21:19:07+00:00,163
6,0,618,16.0,2023-11-16 21:14:51+00:00,1
7,0,618,16.0,2023-11-16 21:19:10+00:00,1
8,0,618,16.0,2023-11-16 21:20:28+00:00,2
9,0,618,16.0,2023-11-16 21:20:46+00:00,1


In [19]:
gossip['startUnix'] = pd.to_datetime(gossip["_time"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
gossip["startUnix"] = pd.to_timedelta(gossip["startUnix"], unit='s').dt.total_seconds().astype(int)
# gossip.head(10)

min_date = gossip["startUnix"].min()
min_time = gossip["_time"].min()

print(min_date, min_time)

1696426715 2023-10-04 13:38:35+00:00


/tmp/ipykernel_43016/1873362235.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossip['startUnix'] = pd.to_datetime(gossip["_time"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


In [21]:
gossip['endUnix'] = pd.to_datetime(gossip["_time"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
gossip["endUnix"] = pd.to_timedelta(gossip["endUnix"], unit='s').dt.total_seconds().astype(int)
# gossip.head(10)

max_date = gossip["endUnix"].max()
max_time = gossip["_time"].max()

print(max_date, max_time)

1700624814 2023-11-22 03:46:54+00:00


/tmp/ipykernel_43016/2093431950.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossip['endUnix'] = pd.to_datetime(gossip["_time"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
